In [351]:
import re
import sys
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords', 'omw-1.4'])
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to /Users/jwang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jwang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jwang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /Users/jwang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/jwang/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [352]:
# load data
category_df = pd.read_csv('disaster_categories.csv')
message_df = pd.read_csv('disaster_messages.csv')
merge_df = category_df.merge(message_df, on='id')

In [353]:
categories = merge_df.categories.str.split(";", expand=True)

row = categories.loc[0]
category_colnames = [re.sub(r"[^a-zA-Z]", "", text) for text in row]
categories.columns = category_colnames

for column in categories:
    categories[column] = categories[column].str[-1]
    categories[column] = pd.to_numeric(categories[column]).astype('Int64')

merge_df = merge_df.drop(['categories', 'genre', 'original'], axis=1)
merge_df = pd.concat([merge_df, categories], axis=1)

In [354]:
# Drop duplicates
merge_df = merge_df.drop_duplicates()
merge_df.head()

,id,message,related,request,offer,aidrelated,medicalhelp,medicalproducts,searchandrescue,security,...,aidcenters,otherinfrastructure,weatherrelated,floods,storm,fire,earthquake,cold,otherweather,directreport
0,2,Weather update - a cold front from Cuba that c...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,1,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [355]:
engine = create_engine('sqlite:///DisasterResponse.db')
merge_df.to_sql('jwang_disaster_pipeline', engine, index=False, if_exists="replace")

26216